In [170]:
import pandas as pd
import numpy as np
from random import randrange
import datetime 
import random

In [21]:
def generate_time(startTime, n):
    current = startTime
    while n >= 0:
        current = current + datetime.timedelta(seconds=randrange(10))
        yield current
        n -= 1

In [85]:
def genearte_dataframe(n_readings, n_sensors, startTime):
    "Creates dataframe of specified number of sensors data"
    
    get_timStamp = [x for x in list(generate_time(startTime,n_readings))]
    get_readings = {f"senor{i}":[round(random.uniform(18, 25),2) for _ in range(n_readings)] for i in list(range(1,n_sensors))}
    get_df = pd.concat([pd.DataFrame(get_timStamp, columns=["timeStamp"]),  pd.DataFrame(get_readings)], axis=1)
    get_df.timeStamp = pd.to_datetime(get_df.timeStamp, infer_datetime_format=True)
    
    return get_df

In [193]:
def result(n_org, n_pred, n_sensor, startTime):
    
    org_df = genearte_dataframe(n_org, n_sensor, startTime)
    pred_df = genearte_dataframe(n_pred, n_sensor, startTime)
    pred_df.set_index("timeStamp", inplace=True)
    pred_df = pred_df.mask(np.random.random(pred_df.shape) < .1)
    pred_df.fillna(0, inplace=True)
    pred_df.reset_index(None, inplace=True)
    result_table = pd.merge_asof(org_df, pred_df, on="timeStamp", direction='nearest')
    result_table.columns = result_table.columns.str.replace("x", "real")
    result_table.columns = result_table.columns.str.replace("y", "pred")
    result_dict = result_table.to_dict(orient="index") # not preferred
    
    return result_table

In [194]:
startTime = datetime.datetime(2020,12,1,0,0,0)
n_org = 5  # number of original readings
n_pred = 15  # number of predicted readings
n_sensor = 5  # number of sensors
result_df = result(n_org, n_pred, n_sensor, startTime)

### Choosing python dataframe as resultant datastructure. 
* Although python dict would perform in search operation with o(1) complexity but as the number of sensors grows this will increase the number of columns which in tern makes the dict slower. 
* Another reason of selecting dataframe, as we have datetime values in our resultant data we can perform a range search like df.loc["2020/01/12 00:00:00" : "2020/01/13 00:00:55"]. 
* Again if we use nested dict then a for loop will make computation even slower. 
* The result_df has unnique sorted timestamp which means accessing rows will be O(m) where m << row_num 

In [196]:
# TEST CASE
startTime = datetime.datetime(2020,12,1,0,0,0)
n_org = 3000  # number of original readings
n_pred = 15000  # number of predicted readings
n_sensor = 500  # number of sensors
result_df = result(n_org, n_pred, n_sensor, startTime)